In [1]:
import sys, os
import tensorflow as tf
import pandas as pd
import numpy as np

Init Plugin
Init Graph Optimizer
Init Kernel


In [5]:
with open("(1) The Hunger Games.txt", 'rb') as file : 
    txt_file = file.readlines()
type(txt_file)

list

"b'they tell me. Sitting at Prims knees, guarding her, is the worlds ugliest cat. Mashed-in nose, half of one ear missing, eyes the color of rotting squash. Prim named him Buttercup, insisting that his muddy yellow coat matched the bright flower. I he hates me. Or at least distrusts me. Even though it was years ago, I think he still remembers how I tried to drown him in a bucket when Prim brought him home. Scrawny kitten, belly swollen with worms, crawling with fleas. The last thing I needed was another mouth to feed. But Prim begged so hard, cried even, I had to let him stay. It turned out okay. My mother got rid of the vermin and hes a born mouser. Even catches the occasional rat. Sometimes, when I clean a kill, I feed Buttercup the entrails. He has stopped hissing at me. Entrails. No hissing. This is the closest we will ever come to love. I swing my legs off the bed and slide into my hunting boots. Supple leather that has molded to my feet. I pull on trousers, a shirt, tuck my long 